In [1]:
import glob
import pandas as pd
import numpy as np

path = 'C:/python/paper/20210506092501'
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    frame = pd.read_csv(filename, index_col = None, header = 0, encoding = 'cp949')
    li.append(frame)
rain = pd.concat(li, axis = 0, ignore_index = True)

rain = rain.drop(['지점', '기온(°C)', '풍향(deg)', '풍속(m/s)', '현지기압(hPa)', '해면기압(hPa)', '습도(%)',
                  '일사(MJ/m^2)', '일조(Sec)'], axis = 1)

rain = rain.drop_duplicates(['일시'])
rain = rain.reset_index(drop = True)

rain['time'] = rain['일시'].str.split(':').str[0]

rain['time'].value_counts().tail(60)

rain = rain.drop_duplicates(['time'], keep = 'last')

rain = rain.reset_index(drop = True)

rain['date'] = rain['time'].str.split(' ').str[0]

rain_list = [0.0]
for i in range(1, len(rain)):
    rain_list.append(rain['누적강수량(mm)'][i] - rain['누적강수량(mm)'][i-1])

len(rain_list)

rain['강수량'] = rain_list

for i in range(0, len(rain)):
    if rain['강수량'][i] < 0:
        rain['강수량'][i] = rain['누적강수량(mm)'][i]

rain_list = rain['강수량'].values.tolist()
rain_list.insert(0, 0)
del rain_list[-1]
rain['강수량'] = rain_list

rain = rain.drop(['일시', '누적강수량(mm)', 'date'], axis = 1)

rain